In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv
/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv
/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv
/kaggle/input/ue20cs344-nam-assignment-2/features.pt


In [3]:
!pip install node2vec

  Preparing metadata (setup.py) ... done
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=c1f17c2ae5a15bb9523fcf25c9de31ee56d8e2d23a9fc55358b6538a022fa399
  Stored in directory: /root/.cache/pip/wheels/41/d8/5d/28571981b2dc5a7003e03031a9f42c9a68f323b8fb1ddbb341
Successfully built node2vec


In [4]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 9.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909879 sha256=370db785203d3af81975a854318c3d32f50bb6a7abb72b91158463f2bb824528
  Stored in directory: /root/.cache/pip/wheels/47/16/75/ac3099e3ea0348d3e241a8230d1fe50b202badaf5d89ae1825
Successfully built torch-geometric


In [61]:
import networkx as nx

import matplotlib.pyplot as plt
from node2vec import Node2Vec
import gensim
from gensim.models import KeyedVectors
import torch
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx
import networkx as nx
import torch_geometric.utils as utils
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv



In [62]:
edgelist = pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv")
sample_submission=pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv')
node_data = pd.read_csv("/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv")
features = torch.load("/kaggle/input/ue20cs344-nam-assignment-2/features.pt")

In [64]:
le = LabelEncoder()
node_data['branch'] = le.fit_transform(node_data['branch'])


In [65]:
G=nx.Graph()
for node in (node_data['id']):
    G.add_node(node)
edges = [(row['source'], row['target']) for _, row in edgelist.iterrows()]
G.add_edges_from(edges) 
nodes = list(G.nodes())
edges = list(G.edges())

In [67]:
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x = features
y = torch.tensor(node_data['branch'].values, dtype=torch.long)
data = Data(x=x, edge_index=edge_index, y=y)
print(data)



Data(x=[19717, 500], edge_index=[2, 44324], y=[19717])


In [68]:


# Define the GCN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(features.shape[1], 16)
        self.conv2 = GCNConv(16, node_data['branch'].nunique())

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model, optimizer and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.NLLLoss()

# Split the train and test mask
train_mask = node_data['train_mask'].values
test_mask = node_data['test_mask'].values

# Train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = criterion(out[train_mask], data.y[train_mask].to(device))
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(200):
    loss = train()
    print('Epoch: {}, Loss: {}'.format(epoch, loss))

# Evaluate the model
model.eval()
_, pred = model(data.x.to(device), data.edge_index.to(device)).max(dim=1)
test_preds = pred[test_mask].detach().cpu().numpy()
test_ids = node_data['id'][test_mask].values

# Decode the branch labels
test_preds = le.inverse_transform(test_preds)




Epoch: 0, Loss: 1.383730411529541
Epoch: 1, Loss: 1.356956124305725
Epoch: 2, Loss: 1.3294552564620972
Epoch: 3, Loss: 1.3026230335235596
Epoch: 4, Loss: 1.275313138961792
Epoch: 5, Loss: 1.2504174709320068
Epoch: 6, Loss: 1.2266085147857666
Epoch: 7, Loss: 1.204660415649414
Epoch: 8, Loss: 1.181833267211914
Epoch: 9, Loss: 1.163300633430481
Epoch: 10, Loss: 1.1434115171432495
Epoch: 11, Loss: 1.1292861700057983
Epoch: 12, Loss: 1.1138572692871094
Epoch: 13, Loss: 1.094181776046753
Epoch: 14, Loss: 1.0801022052764893
Epoch: 15, Loss: 1.0641422271728516
Epoch: 16, Loss: 1.0524251461029053
Epoch: 17, Loss: 1.0411248207092285
Epoch: 18, Loss: 1.0265655517578125
Epoch: 19, Loss: 1.0183347463607788
Epoch: 20, Loss: 1.0018541812896729
Epoch: 21, Loss: 0.9901595115661621
Epoch: 22, Loss: 0.9799154996871948
Epoch: 23, Loss: 0.971259593963623
Epoch: 24, Loss: 0.9612054824829102
Epoch: 25, Loss: 0.9468568563461304
Epoch: 26, Loss: 0.9317088723182678
Epoch: 27, Loss: 0.9250091314315796
Epoch: 28,

In [69]:
# Create the submission file
submission_df = pd.DataFrame({'id': test_ids, 'branch': test_preds})
print(submission_df.head())
submission_df.to_csv('submission.csv',index=False)

   id branch
0   4    ECE
1  22    EEE
2  29    ECE
3  32    ECE
4  34    CSE
